In [1]:
# 1. 데이터 읽기
import pandas as pd

gyeongju_poi = pd.read_csv('gyeongju_poi2.csv', index_col=False)

gyeongju_poi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1623 entries, 0 to 1622
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   keyword              1623 non-null   object 
 1   address_name         1623 non-null   object 
 2   category_group_code  1623 non-null   object 
 3   category_group_name  1623 non-null   object 
 4   category_name        1623 non-null   object 
 5   distance             0 non-null      float64
 6   id                   1623 non-null   int64  
 7   phone                553 non-null    object 
 8   place_name           1623 non-null   object 
 9   place_url            1623 non-null   object 
 10  road_address_name    932 non-null    object 
 11  x                    1623 non-null   float64
 12  y                    1623 non-null   float64
dtypes: float64(3), int64(1), object(9)
memory usage: 165.0+ KB


In [2]:
keywords = set()
for keyword in gyeongju_poi['keyword'].unique():
    for k in keyword.split(','):
        keywords.add(k)
keywords = list(keywords)

keywords

['체험', '가볼만한 곳', '감성', '여행', '맛집']

In [3]:
groups = []

for category in gyeongju_poi['category_group_name'].unique():
    groups.append(category)

groups

['문화시설', '관광명소', '카페']

In [4]:
gyeongju_poi[['id', 'place_name', 'keyword', 'category_group_name']].head(1)

,id,place_name,keyword,category_group_name
0,1394788185,송대말등대 빛체험전시관,체험,문화시설


In [5]:
from tqdm.notebook import tqdm
gyeongju_poi['content'] = ''

for idx, row in tqdm(gyeongju_poi.iterrows()):
    content = (row['keyword']+','+row['category_group_name']).replace(' ', '').replace(',', ' ')
    gyeongju_poi.loc[idx, 'content'] = content

0it [00:00, ?it/s]

In [6]:
gyeongju_poi[['id', 'place_name', 'keyword', 'category_group_name', 'content']].head(2)

,id,place_name,keyword,category_group_name,content
0,1394788185,송대말등대 빛체험전시관,체험,문화시설,체험 문화시설
1,11256465,오류고아라해변,"여행,가볼만한 곳",관광명소,여행 가볼만한곳 관광명소


In [7]:
gyeongju_poi.loc[0, 'content']

'체험 문화시설'

In [8]:
gyeongju_poi.to_csv('gyeongju_poi_content.csv', index=False)

In [9]:
gyeongju_poi = pd.read_csv('gyeongju_poi_content.csv', index_col=False)

gyeongju_poi.head(1)

,keyword,address_name,category_group_code,category_group_name,category_name,distance,id,phone,place_name,place_url,road_address_name,x,y,content
0,체험,경북 경주시 감포읍 오류리 583-3,CT1,문화시설,"문화,예술 > 문화시설 > 전시관",NaN,1394788185,NaN,송대말등대 빛체험전시관,http://place.map.kakao.com/1394788185,경북 경주시 감포읍 감포로 226-19,129.511359,35.807336,체험 문화시설


In [10]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1, 2))
content_mat = count_vect.fit_transform(gyeongju_poi['content'])
content_mat.shape

(1623, 22)

In [12]:
count_vect.get_feature_names()

['가볼만한곳',
 '가볼만한곳 관광명소',
 '가볼만한곳 여행',
 '감성',
 '감성 감성',
 '감성 맛집',
 '감성 카페',
 '관광명소',
 '맛집',
 '맛집 감성',
 '맛집 맛집',
 '맛집 여행',
 '맛집 카페',
 '문화시설',
 '여행',
 '여행 가볼만한곳',
 '여행 맛집',
 '여행 여행',
 '여행 카페',
 '체험',
 '체험 문화시설',
 '카페']

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

content_sim = cosine_similarity(content_mat, content_mat)
print(content_sim.shape)
print(content_sim[:2])

(1623, 1623)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]


In [14]:
ex_poi = gyeongju_poi.copy(deep = True)
ex_poi['nl_cosine'] = content_sim[0]

In [15]:
ex_poi[['id', 'place_name', 'nl_cosine']].head(5)

,id,place_name,nl_cosine
0,1394788185,송대말등대 빛체험전시관,1.0
1,11256465,오류고아라해변,0.0
2,8223627,나정고운모래해변,0.0
3,10714629,전촌솔밭해변,0.0
4,2087577487,감포해국길,0.0


In [17]:
import numpy as np

np.save('nl_content_sim.npy', content_sim)

In [18]:
import numpy as np

np_file = np.load(file='nl_content_sim.npy')
np_file[0][:5]

array([1., 0., 0., 0., 0.])